# Missionaries and Cannibals

This will be another water-jug like problem, but a bit more complex.
Feel free to skim this tutorial.

We are going to model and solve the missionary and cannibal (MAC) problem. 
It is described below.

> Three missionaries and three cannibals come to a river. 
There is a boat on their bank of the river that can be used by either one or two persons at a time. 
This boat must be used to cross the river in such a way that cannibals never outnumber missionaries on either bank of the river (although cannibals can be alone on one bank). 
How do the missionaries and cannibals successfully cross the river?

Eaters is competitive and requires fast intelligent responses to the current situation
Missionaries and Cannibals can be solved in however long time, can be solved in trial and error through search

![mac graph](./img/5_mac-graph.png)

We want to create rules that:
1. Initialze state
2. Propose and apply operators
3. Check if we are at the goal
4. Check if we have failed

Why not just only propose operators that won't lead us to a failure state? 
Soar tutorial says, "
However that is moving an aspect of the problem into the problem space and requires some problem 
solving to determine what the conditions of the proposal should be. 
We will see in Part V how Soar can learn to rules that avoid proposing operators when they will lead to failure."

### State representation
We have to only represent the number of cannibals / missionaries on each side of the bank.
And which side the boat is on.
Don't have to represent people in a boat traveling, as that is implicit through operator application.

3 ways to model this
[copy state 1]
[copy state 2 representation & graph]
[copy state 3 representation & graph]
Last two are valid but we will use #2 because it most closely relates to the physical structure of the problem.

### Initialization

We are going to add: 
- the initialize state (3 missionaries and cannibals on the left bank)
- A utility state `other-bank` (used in our operator's application)
- A utility state `desired` (used to halt when we find the goal state)

In [ ]:
mac_initialization = """
sp {mac*propose*initialize-mac 
    (state <s> ^superstate nil
              -^name)
-->
    (<s> ^operator <o> +)
    (<o> ^name initialize-mac)
}

sp {mac*apply*initialize-mac
    (state <s> ^operator.name initialize-mac) 
-->
    (<s> ^name mac
         ^left-bank <l> 
         ^right-bank <r> 
         ^desired <d>)
    (<r> ^missionaries 0 
         ^cannibals    0
         ^boat         0
         ^other-bank  <l>) 
    (<l> ^missionaries 3
         ^cannibals    3 
         ^boat         1 
         ^other-bank  <r>)
    (<d> ^right-bank <dr>) 
    (<dr> ^missionaries 3
          ^cannibals    3 
          ^boat         1)}
"""

### Operators

Operators role is to move 1 or 2 people across the bank. 
Three ways to do this
- Move 1 missionary *or* 1 cannibal
- Move 2 missionaries *or* 2 cannibals
- Move 1 missionary *and* 1 cannibal

The rules are straightforward: 
1. If there are 1+ cannibal or missionary on the boat's bank, 
    then propose moving that cannibal/missionary
2. If there are 2+ cannibal or missionary on the boat's bank, 
    then propose moving 2 cannibals or 2 missionaries
3. If there are 1+ cannibal and 1+ missionary on the boat's bank,
    then propose moving 1 missionary and 1 cannibal

We'll have one generic operator application that will take:
- Type of person being moved (`<< cannibals missionaries >>`)
- Number of `types` being moved (`1` or `2`)
- And the bank we are coming from (`<< left-bank right-bank >>`)

#### Proposal

We'll go over one of the rules together, the rest will follow this same sort of format.

```
sp {mac*propose*operator*move-mac-boat1...
```

*Match on either the `left-bank` or `right-bank` and store that into `<bank>`*

```
               ^<< left-bank right-bank >> <bank>) 
```

*Match on either `cannibals` or `missionaries` where the value if > 0
and store that into `<type>`*

```
    (<bank> ^{ << cannibals missionaries >> <type> } > 0
```
*Ensure the boat is on this side of the bank*
```
            ^boat 1) 
-->
    (<s> ^operator <o> + =) 
    (<o> ^name move-mac-boat
```
*Add the parameters that we discussed above onto the proposal.*
```
         ^bank <bank> 
         ^<type> 1 
         ^boat 1 
         ^types 1)}
```

In [ ]:
mac_proposals = """
# Move a single cannibal or missionary
sp {mac*propose*operator*move-mac-boat1 
    (state <s> ^name mac
               ^<< right-bank left-bank >>     <bank>) 
    (<bank>    ^{ << cannibals missionaries >> <type> } > 0
               ^boat 1) 
-->
    (<s> ^operator <o> + =) 
    (<o> ^name   move-mac-boat
         ^bank   <bank> 
         ^<type> 1 
         ^boat   1 
         ^types  1)
}

# Move two of one type
sp {mac*propose*operator*move-mac-boat2 
    (state <s> ^name mac
               ^ << right-bank left-bank >> <bank>) 
    (<bank>    ^{ << cannibals missionaries >> <type> } > 1
               ^boat 1) 
-->
    (<s> ^operator <o> + =) 
    (<o> ^name   move-mac-boat
         ^bank   <bank> 
         ^<type> 2 
         ^boat   1 
         ^types  1)
}

# Move one missionary and one cannibal
sp {mac*propose*operator*move-mac-boat11 
    (state <s> ^name mac
               ^ << right-bank left-bank >> <bank>) 
    (<bank>    ^missionaries > 0
               ^cannibals > 0
               ^boat 1) 
-->
    (<s> ^operator <o> + =) 
    (<o> ^name         move-mac-boat
         ^bank         <bank> 
         ^missionaries 1 
         ^cannibals    1 
         ^boat         1
         ^types        2)
}
"""

### Application 

In the application, we have to decrease the number of missionaries/cannibals leaving the
*from* bank and increase the number entering the *to* bank.
We also have to flip the boat count variable from 1 to 0 on the *from* bank and 0 to 1 on the *to* bank.

Read the following example where we check for the operator named `move-mac-boat`
where we move *a single* cannibal.
We will generalize this rule in a minute.

```
sp {apply*move-mac-boat*one*cannibal 
    (state <s> ^operator <o>)
    (<o> ^name      move-mac-boat
         ^cannibals 1
         ^bank      <from-bank>)
    (<from-bank> ^cannibals  <from-bank-num>
                 ^other-bank <to-bank>) 
    (<to-bank>   ^cannibals  <to-bank-num>)
-->
    (<from-bank> ^cannibals <from-bank-num> -
                         (- <from-bank-num> 1)) 
    (<to-bank>   ^cannibals <to-bank-num> -
                         (+ <to-bank-num> 1))}
```


You can see how the `other-bank` utility state comes in handy here.
Now we can take a big step to generically move any type from one bank to the other.

In [ ]:
mac_apply = """
sp {apply*move-mac-boat 
    (state <s> ^operator <o>) 
    (<o> ^name move-mac-boat
         ^{ << missionaries cannibals boat >> <type> } <number>
         ^bank <from-bank>)
    (<from-bank>  ^<type>     <from-bank-num>
                  ^other-bank <to-bank>) 
    (<to-bank>    ^<type>     <to-bank-num>)
-->
    (<from-bank> ^<type> <from-bank-num> -
                      (- <from-bank-num> <number>)) 
    (<to-bank> ^<type> <to-bank-num> -
                    (+ <to-bank-num> <number>))}
"""

If we are moving 1 cannibal and 1 missionary, three operator applications will fire in parallel:
1. Increment value `boat` from `from-bank` and increment on `to-bank`
1. Increment value `missionaries` from `from-bank` and increment on `to-bank`
1. Increment value `cannibals` from `from-bank` and increment on `to-bank`

### Monitoring

Here are some rules we'll use to log state changes.

In [ ]:
mac_monitor = """
sp {monitor*move-mac-boat 
    (state <s> ^operator <o>) 
    (<o> ^name move-mac-boat
         ^{ << cannibals missionaries >> <type> } <number>) 
-->
    (write | Move | <number> | | <type>)
}

sp {monitor*state*left 
    (state <s> ^name       mac
               ^left-bank  <l>
               ^right-bank <r>) 
    (<l> ^missionaries <ml>
         ^cannibals    <cl>
         ^boat         1)
    (<r> ^missionaries <mr>
         ^cannibals <cr>
         ^boat 0) 
-->
    (write(crlf)|M:|<ml>|,C:|<cl>|B~~~ | |M:|<mr>|,C:|<cr>| |)
}

sp {monitor*state*right 
    (state <s> ^name       mac
               ^left-bank  <l>
               ^right-bank <r>) 
    (<l> ^missionaries <ml>
         ^cannibals    <cl>
         ^boat         0)
    (<r> ^missionaries <mr>
         ^cannibals    <cr>
         ^boat         1) 
-->
    (write(crlf)|M:|<ml>|,C:|<cl>| ~~~B| |M:|<mr>|,C:|<cr>| |)
}
"""

### Desired State Recognition

We're going to write a rule that halts our program if we reach the desired state.
Remember that the desired state was added during state initialization.

This rule will be written to handle the desired state being changed later.

For example, if you wanted to change the desired final side of the bank, or missionary/cannibal count, 
you would not have to rewrite this rule.

In [ ]:
mac_desired_state = """
sp {mac*detect*state*success 
    (state <s> ^desired <d>
               ^<side> <ls>)
    # Throw the desired number of missionaries/cannibals into `<m>` and `<c>`
    (<d>   ^{ << right-bank left-bank >> <side> } <dls>) 
    (<dls> ^missionaries <m>
           ^cannibals    <c>) 
    # Check if the 
    (<ls>  ^missionaries <m>
           ^cannibals    <c>)    
-->
    (write (crlf) |The problem has been solved.|) (halt)}
"""

If we run the program, we will eventually hit the desired state. 

However, this problem has a failure condition, so let's write a rule to halt if we fail.

### Failure State Recognition

If the cannibals outnumber the missionaries on one bank of the river, halt.

In [ ]:
mac_failure_state = """
sp {mac*evaluate*state*failure*more*cannibals 
    (state <s> ^desired <d>
               ^<< right-bank left-bank >> <bank>) 
    (<bank> ^missionaries { <n> > 0 }
            ^cannibals          > <n>)
-->
    (write (crlf) |The problem has failed.|) (halt)}
"""

More often than not, we will hit a failure state. 
The tutorial pages 156-159 go over undoing failure states. 

A good exercise would be to implement undoing of states.
After, you can apply the lessons learned in lookahead search to add planning and chunking to this problem (solution is covered in pages 173-176).